# **Team**

> Balestrieri Niccolò - 10936955 <br>
  Bertogalli Andrea - 10702303 <br>
  Cavalieri Francesco - 11020855    
  Tombini Nicolò - 10912627


# Competition among bidding algotithms

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import stats as scipystats
from scipy import optimize

np.random.seed(42)

## Auction definition

In [ ]:
class Auction:
    def __init__(self, *args, **kwargs):
      raise NotImplementedError

    def get_winners(self, bids):
      raise NotImplementedError

    def get_payments_per_click(self, winners, values, bids):
      raise NotImplementedError

    def round(self, bids):
      winners, values = self.get_winners(bids)
      payments_per_click = self.get_payments_per_click(winners, values, bids)
      return winners, payments_per_click

In [ ]:
class GeneralizedFirstPriceAuction(Auction):
    def __init__(self, ctrs, slots):
        self.ctrs = ctrs
        self.n_adv = len(self.ctrs)
        self.slots = slots

    def get_winners(self, bids):
        adv_values = self.ctrs*bids
        adv_ranking = np.argsort(adv_values)[::-1]
        winners = adv_ranking[:self.slots]
        return winners, adv_values[winners]

    def get_payments_per_click(self, winners, values, bids):
        payment = bids[winners]
        return payment.round(2)

## Agents definition

In [ ]:
class Agent:
  def __init__(self):
    raise NotImplementedError

  def pull_arm(self):
    raise NotImplementedError

  def update(self, r_t):
    raise NotImplementedError

In [ ]:
class HedgeAgent(Agent):
    def __init__(self, K, learning_rate):
        self.K = K
        self.learning_rate = learning_rate
        self.weights = np.ones(K)
        self.x_t = np.ones(K)/K
        self.a_t = None
        self.t = 0

    def pull_arm(self):
        self.x_t = self.weights/sum(self.weights)
        self.a_t = np.random.choice(np.arange(self.K), p=self.x_t)
        return self.a_t

    def update(self, l_t):
        self.weights *= np.exp(-self.learning_rate*l_t)
        self.t += 1

In [ ]:
class UCBLike(Agent):
  def __init__(self, K, T, range=1):
    self.K = K
    self.T = T
    self.range = range
    self.a_t = None
    self.average_profit = np.zeros(self.K)
    self.average_cost = np.zeros(self.K)
    self.profit_ucbs = np.zeros(self.K)
    self.cost_ucbs = np.zeros(self.K)
    self.N_pulls = np.zeros(self.K)
    self.arm_probabilities = np.zeros(self.K)
    self.t = 0

  def pull_arm(self, rho):
    if self.t < self.K:
      self.a_t = self.t
    else:
      self.profit_ucbs = self.average_profit + self.range*np.sqrt(2*np.log(self.T)/self.N_pulls)
      self.cost_ucbs = self.average_cost - self.range*np.sqrt(2*np.log(self.T)/self.N_pulls)

      c = -self.profit_ucbs
      A_ub = [self.cost_ucbs]
      b_ub = [rho]
      A_eq = [np.ones(self.K)]
      b_eq = [1]
      res = optimize.linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=(0,1))
      gamma = res.x
      self.arm_probabilities = gamma

      bid_index = np.random.choice(self.K, p=gamma)
      self.a_t = bid_index

    return self.a_t

  def update(self, f_t, c_t):
    self.N_pulls[self.a_t] += 1
    self.average_profit[self.a_t] += (f_t - self.average_profit[self.a_t])/self.N_pulls[self.a_t]
    self.average_cost[self.a_t] += (c_t - self.average_cost[self.a_t])/self.N_pulls[self.a_t]
    self.t += 1

## Strategies definitions

In [ ]:
class BiddingAgent:
  def __init__(self):
    raise NotImplementedError

  def bid(self):
    raise NotImplementedError

  def update(self):
    raise NotImplementedError

In [ ]:
class HedgeMultiplicativePacingAgent(BiddingAgent):
    def __init__(self, bids_set, valuation, budget, T, eta):
        self.bids_set = bids_set
        self.K = len(bids_set)
        self.hedge = HedgeAgent(self.K, np.sqrt(np.log(self.K)/T))
        self.valuation = valuation
        self.budget = budget
        self.eta = eta
        self.T = T
        self.rho = self.budget/self.T
        self.lmbd = 1
        self.t = 0

    def bid(self):
        if self.budget < 1:
            return 0
        return self.bids_set[self.hedge.pull_arm()]

    def update(self, f_t, c_t, m_t):
        f_t_full = np.array([(self.valuation-b)*int(b >= m_t) for b in self.bids_set])
        c_t_full = np.array([b*int(b >= m_t) for b in self.bids_set])
        L = f_t_full - self.lmbd*(c_t_full-self.rho)
        range_L = 2+(1-self.rho)/self.rho
        self.hedge.update((2-L)/range_L)
        self.lmbd = np.clip(self.lmbd-self.eta*(self.rho-c_t), a_min=0, a_max=1/self.rho)
        self.budget -= c_t

In [ ]:
class TruthfulMultiplicativePacingAgent(BiddingAgent):
    def __init__(self, valuation, budget, T, eta):
        self.valuation = valuation
        self.budget = budget
        self.eta = eta
        self.T = T
        self.rho = self.budget/self.T
        self.lmbd = 1
        self.t = 0

    def bid(self):
        if self.budget < 1:
            return 0
        return self.valuation/(self.lmbd+1)

    def update(self, f_t, c_t):
        self.lmbd = np.clip(self.lmbd-self.eta*(self.rho-c_t),
                            a_min=0, a_max=1/self.rho)
        self.budget -= c_t

In [ ]:
class UCBLikeMultiplicativePacingAgent(BiddingAgent):
  def __init__(self, bids_set, valuation, budget, T, eta):
    self.bids_set = bids_set
    self.K = len(bids_set)
    self.T = T
    self.ucb = UCBLike(self.K, self.T, range=0.01)
    self.budget = budget
    self.eta = eta
    self.rho = self.budget/self.T
    self.lmbd = 1
    self.t = 0
    self.valuation = valuation

  def bid(self):
    bid = self.bids_set[self.ucb.pull_arm(self.rho)]
    if self.budget < bid:
        bid = self.budget
    return bid

  def update(self, f_t, c_t, my_bid):
    self.ucb.update(f_t, c_t)
    self.budget -= c_t

  def get_arm_probabilities(self):
    return self.ucb.arm_probabilities

## Trials

In [ ]:
class Logs:
    def __init__(self):
      self.records = {
          'payment_history':[],
          'reward_history' : [],
          'bid_history': [],
          'expected_clairvoyant_utilities':[],
          'expected_clairvoyant_bids' : [],
          'agent_names': [],
      }

    def add_log(self, field, value):
      self.records[field].append(value)

    def get_logs(self):
      return self.records

    def get_logs_count(self):
      return len(self.records['payment_history'])

In [ ]:
class InteractionSettings:
  def __init__(self):
    self.SLOT = 2
    self.TRIALS = 5
    self.BUDGET = 100
    self.USERS = 600
    self.AGENTS = []
    self.VALUATIONS = [0.3,0.5]
    self.K = 10
    self.BIDS = np.linspace(0, 1, self.K)
    self.clairvoyant_k = 50


  def add_agent(self, agent, name):
    self.AGENTS.append({
        "Agent_name": name,
        "Agent": agent
    })

In [ ]:
def max_except_column(arr, n_slots):
    results = np.zeros_like(arr)
    for row_idx in range(arr.shape[0]):
        row = arr[row_idx]

        for col_idx in range(arr.shape[1]):
            included_values = np.concatenate((row[:col_idx], row[col_idx + 1:]))
            sorted_values = np.sort(included_values)[::-1]

            if n_slots <= len(sorted_values):
                nth_largest = sorted_values[n_slots - 1]
            else:
                nth_largest = sorted_values[-1]

            results[row_idx, col_idx] = nth_largest

    return results

In [ ]:
def bidding_interaction_simulation(settings, logs):
  for trial in tqdm(range(settings.TRIALS), desc="Trial n°"):
    settings = InteractionSettings()

    for v in settings.VALUATIONS:
        # Truthful
        settings.add_agent(
            name = f"Thrutful_v={v}",
            agent = TruthfulMultiplicativePacingAgent(v, settings.BUDGET, settings.USERS, 1/np.sqrt(settings.USERS))
        )

        # Hedge
        settings.add_agent(
            name = f"Hedge_v={v}",
            agent =HedgeMultiplicativePacingAgent(settings.BIDS, v,settings.BUDGET, settings.USERS, 1/np.sqrt(settings.USERS))
        )

        # UCBLike
        settings.add_agent(
            name = f"UCBLike_v={v}",
            agent =UCBLikeMultiplicativePacingAgent(settings.BIDS, v, settings.BUDGET, settings.USERS, 1/np.sqrt(settings.USERS))
        )
    ctrs = np.ones(len(settings.AGENTS))

    ## AUCTION GAME
    auction = GeneralizedFirstPriceAuction(ctrs, settings.SLOT)
    np.random.seed(trial)

    reward_history = []
    payment_history = []
    bid_history = []

    for u in range(settings.USERS):
      f_t, c_t = 0, 0
      # Agent - Auction Interaction
      bids = [agent["Agent"].bid() for agent in settings.AGENTS]
      bid_history.append(bids)
      winners, _ = auction.get_winners(bids)

      min_winnig_bid = min([bids[i] for i in winners])

      rewards = [0 for x in range(len(settings.AGENTS))]
      payments = [0 for x in range(len(settings.AGENTS))]

      for i in winners:
        rewards[i] = settings.AGENTS[i]["Agent"].valuation - bids[i]
        payments[i] = bids[i]

      payment_history.append(payments)
      reward_history.append(rewards)

      for  i in range(len(settings.AGENTS)):
        if isinstance(settings.AGENTS[i]["Agent"], TruthfulMultiplicativePacingAgent):
          settings.AGENTS[i]["Agent"].update(f_t=rewards[i] , c_t=payments[i])
        elif isinstance(settings.AGENTS[i]["Agent"], HedgeMultiplicativePacingAgent):
          settings.AGENTS[i]["Agent"].update(f_t=rewards[i] , c_t=payments[i] , m_t = min_winnig_bid)
        else:
           settings.AGENTS[i]["Agent"].update(f_t = rewards[i], c_t = payments[i], my_bid = bids[i])

    payment_history = np.array(payment_history)
    reward_history = np.array(reward_history)
    bid_history= np.array(bid_history)

    clairvoyant_bids_set = np.linspace(0, 1, settings.clairvoyant_k)
    n_agents = len(settings.AGENTS)
    time = bid_history.shape[0]
    n_bids = clairvoyant_bids_set.shape[0]
    rho = settings.BUDGET / time

    max_bids = max_except_column(bid_history, settings.SLOT)
    win_probabilities = np.zeros((n_agents, n_bids))
    for i in range(n_agents):
        for j, b in enumerate(clairvoyant_bids_set):
          win_probabilities[i, j] = sum(b > max_bids[:,i]) / time

    expected_clairvoyant_utilities = np.zeros((n_agents, time))
    expected_clairvoyant_bids = np.zeros((n_agents, time))

    for i in range(n_agents):
      # Associa una valutazione ciclicamente
      valuation = settings.VALUATIONS[i % len(settings.VALUATIONS)]
      c = -(valuation - clairvoyant_bids_set) * win_probabilities[i]

      # Linear Program
      A_ub = [clairvoyant_bids_set * win_probabilities[i]]
      b_ub = [rho]
      A_eq = [np.ones(len(clairvoyant_bids_set))]
      b_eq = [1]
      res = optimize.linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=(0, 1))

      gamma = res.x
      expected_clairvoyant_utilities[i] = [-res.fun for u in range(time)]
      expected_clairvoyant_bids[i] = [sum(clairvoyant_bids_set * gamma * win_probabilities[i]) for u in range(time)]

    agent_names = [agent["Agent_name"] for agent in settings.AGENTS]

    logs.add_log("payment_history", payment_history)
    logs.add_log("reward_history", reward_history)
    logs.add_log("bid_history", bid_history)
    logs.add_log("expected_clairvoyant_utilities", expected_clairvoyant_utilities)
    logs.add_log("expected_clairvoyant_bids", expected_clairvoyant_bids)
    logs.add_log("agent_names", agent_names)

In [ ]:
settings, logs = InteractionSettings(), Logs()
bidding_interaction_simulation(settings, logs)

Trial n°: 100%|██████████| 5/5 [00:08<00:00,  1.77s/it]


In [ ]:
logs_data = logs.get_logs()

reward_history = np.mean(np.array(logs_data["reward_history"]), axis = 0)
expected_clairvoyant_utilities = np.mean(np.array(logs_data["expected_clairvoyant_utilities"]), axis = 0)
payment_history = np.mean(np.array(logs_data["payment_history"]), axis = 0)
bid_history = np.mean(np.array(logs_data["bid_history"]), axis = 0)
expected_clairvoyant_bids = np.mean(np.array(logs_data["expected_clairvoyant_bids"]), axis = 0)

reward_history_std = np.std(np.array(logs_data["reward_history"]), axis = 0)
expected_clairvoyant_utilities_std = np.std(np.array(logs_data["expected_clairvoyant_utilities"]), axis = 0)
payment_history_std = np.std(np.array(logs_data["payment_history"]), axis = 0)
bid_history_std = np.std(np.array(logs_data["bid_history"]), axis = 0)
expected_clairvoyant_bids_std = np.std(np.array(logs_data["expected_clairvoyant_bids"]), axis = 0)

agent_names = logs_data["agent_names"][0]

regret = expected_clairvoyant_utilities - reward_history.T
cumulative_regret = np.cumsum(regret.T, axis=0)
cumulative_payment = np.cumsum(payment_history, axis=0)
cumulative_rewards = np.cumsum(reward_history, axis=0)
sorted_indices_rewards = np.argsort(cumulative_rewards[-1, :])[::-1]

In [ ]:
def generate_colormap_colors(n):
    colormap = plt.get_cmap('tab10')  # Ottieni il colormap
    colors = colormap.colors  # Estrarre i colori
    return [f'rgba({int(color[0] * 255)}, {int(color[1] * 255)}, {int(color[2] * 255)}, 1)' for color in colors[:n]]

# Generazione dei colori basata sul numero di agenti
colors = generate_colormap_colors(len(agent_names))

# Grafico dei rewards
fig_rewards = go.Figure()
for idx, i in enumerate(sorted_indices_rewards):
    std_low = cumulative_rewards[:, i] - reward_history_std[:, i]
    std_high = cumulative_rewards[:, i] + reward_history_std[:, i]
    fig_rewards.add_trace(go.Scatter(
        x=list(range(settings.USERS)),
        y=cumulative_rewards[:, i],
        mode='lines',
        name=agent_names[i],
        line=dict(color=colors[idx])  # Usa un colore unico per ogni agente
    ))
    fig_rewards.add_trace(go.Scatter(
        x=list(range(settings.USERS)) + list(range(settings.USERS))[::-1],
        y=list(std_high) + list(std_low[::-1]),
        fill='toself',
        fillcolor=colors[idx].replace('1)', '0.2)'),  # Aggiungi trasparenza per lo sfondo
        line=dict(color='rgba(0,0,0,0)'),  # Linea trasparente
        hoverinfo="skip",
        showlegend=False
    ))

fig_rewards.update_layout(
    title='Cumulative Rewards with Standard Deviation',
    xaxis_title='Users',
    yaxis_title='Reward'
)

# Grafico dei payments
fig_payments = go.Figure()
for idx, i in enumerate(sorted_indices_rewards):
    std_low = cumulative_payment[:, i] - payment_history_std[:, i]
    std_high = cumulative_payment[:, i] + payment_history_std[:, i]
    fig_payments.add_trace(go.Scatter(
        x=list(range(settings.USERS)),
        y=cumulative_payment[:, i],
        mode='lines',
        name=agent_names[i],
        line=dict(color=colors[idx])  # Usa un colore unico per ogni agente
    ))
    fig_payments.add_trace(go.Scatter(
        x=list(range(settings.USERS)) + list(range(settings.USERS))[::-1],
        y=list(std_high) + list(std_low[::-1]),
        fill='toself',
        fillcolor=colors[idx].replace('1)', '0.2)'),  # Aggiungi trasparenza per lo sfondo
        line=dict(color='rgba(0,0,0,0)'),  # Linea trasparente
        hoverinfo="skip",
        showlegend=False
    ))
fig_payments.add_trace(go.Scatter(
    x=list(range(settings.USERS)),
    y=[settings.BUDGET] * settings.USERS,
    mode='lines',
    name='Budget',
    line=dict(dash='dash', color='red')
))

fig_payments.update_layout(
    title='Cumulative Payments with Standard Deviation',
    xaxis_title='Users',
    yaxis_title='Payment'
)

# Grafico del regret
fig_regret = go.Figure()
for idx, i in enumerate(sorted_indices_rewards):
    std_low = cumulative_regret[:, i] - reward_history_std[:, i]
    std_high = cumulative_regret[:, i] + reward_history_std[:, i]
    fig_regret.add_trace(go.Scatter(
        x=list(range(settings.USERS)),
        y=cumulative_regret[:, i],
        mode='lines',
        name=agent_names[i],
        line=dict(color=colors[idx])  # Usa un colore unico per ogni agente
    ))
    fig_regret.add_trace(go.Scatter(
        x=list(range(settings.USERS)) + list(range(settings.USERS))[::-1],
        y=list(std_high) + list(std_low[::-1]),
        fill='toself',
        fillcolor=colors[idx].replace('1)', '0.2)'),  # Aggiungi trasparenza per lo sfondo
        line=dict(color='rgba(0,0,0,0)'),  # Linea trasparente
        hoverinfo="skip",
        showlegend=False
    ))

fig_regret.update_layout(
    title='Regret History with Standard Deviation',
    xaxis_title='Users',
    yaxis_title='Regret'
)

fig_rewards.show()
fig_payments.show()
fig_regret.show()
